In [1]:
import pandas as pd

In [2]:
customer_summary = pd.read_csv('customer_summary.csv') 
#transactions_dataset = pd.read_csv('data/transactions_dataset.csv', sep=";")
relationships = pd.read_csv('data/sales_client_relationship_dataset.csv')

In [3]:
customer_summary = customer_summary.merge(relationships, on='client_id', how='left')

In [4]:
customer_summary["quali_relation"] = customer_summary["quali_relation"].map({"Agreeable client": "1 Email",
                                                                              "Demanding client": "2 Phone calls",
                                                                                "Difficult client": "1 Visit"})
customer_summary.rename(columns={"quali_relation": "effort_required"}, inplace=True)
customer_summary["cost"] = customer_summary["effort_required"].map({"1 Email": 0,
                                                                    "2 Phone calls": 160,
                                                                    "1 Visit": 320})

In [ ]:
top_1_percent_threshold = customer_summary['total_monetary'].quantile(0.99)
top_1_percent_spenders_df = customer_summary[customer_summary['total_monetary'] >= top_1_percent_threshold]
top_1_percent_spenders = top_1_percent_spenders_df['client_id'].tolist()
#top_1_percent_spenders_transactions = transactions_dataset[transactions_dataset['client_id'].isin(top_1_percent_spenders)]

In [ ]:
top_1_percent_spenders_df['status'] = top_1_percent_spenders_df['churner'].map({True: 'churned', False: 'active'})
top_1_percent_spenders_df.drop(columns=['churner', "Unnamed: 0", "dynamic_threshold"], inplace=True)
top_1_percent_spenders_df = top_1_percent_spenders_df.rename(columns={"total_monetary": "total_spendings",
                                                                      "frequency": "total_transactions",
                                                                      "avg_monetary": "avg_transaction_value",
                                                                      "recency": "days_since_last_transaction",
                                                                      "avg_time_gap": "avg_days_between_transactions"})

C:\Users\charl\AppData\Local\Temp\ipykernel_9516\3409040961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1_percent_spenders_df['status'] = top_1_percent_spenders_df['churner'].map({True: 'churned', False: 'active'})
C:\Users\charl\AppData\Local\Temp\ipykernel_9516\3409040961.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1_percent_spenders_df.drop(columns=['churner', "Unnamed: 0", "dynamic_threshold"], inplace=True)


In [ ]:
# Convert 'last_purchase' to datetime
top_1_percent_spenders_df['last_purchase'] = pd.to_datetime(top_1_percent_spenders_df['last_purchase'])

# Calculate the number of months each customer has been active
top_1_percent_spenders_df['months_active'] = (top_1_percent_spenders_df['last_purchase'].dt.year - 2017) * 12 + top_1_percent_spenders_df['last_purchase'].dt.month - 9

# Calculate the average monthly spend
top_1_percent_spenders_df['avg_monthly_spend'] = top_1_percent_spenders_df['total_spendings'] / top_1_percent_spenders_df['months_active']

# Round the average monthly spend
top_1_percent_spenders_df['avg_monthly_spend'] = top_1_percent_spenders_df['avg_monthly_spend'].round(2)

top_1_percent_spenders_df["potential_monthly_profit"] = top_1_percent_spenders_df["avg_monthly_spend"] * 0.9 - (top_1_percent_spenders_df["cost"])

top_1_percent_spenders_df['potential_monthly_profit'] = top_1_percent_spenders_df['potential_monthly_profit'].round(2)

# Round the average days between transactions
top_1_percent_spenders_df['avg_days_between_transactions'] = top_1_percent_spenders_df['avg_days_between_transactions'].round(2)

In [ ]:
top_1_percent_spenders_df

,client_id,last_purchase,total_spendings,avg_transaction_value,total_transactions,days_since_last_transaction,avg_days_between_transactions,effort_required,cost,status,months_active,avg_monthly_spend,potential_monthly_profit
1,14,2019-09-20,3103187,243.158351,505,2,1.44,1 Email,0,active,24,129299.46,116369.51
37,753,2019-09-19,997376,196.954203,284,3,2.55,1 Visit,320,active,24,41557.33,37081.60
292,4696,2019-09-20,1152072,120.484416,492,2,1.48,1 Visit,320,active,24,48003.00,42882.70
434,6839,2019-09-17,1852492,161.578021,455,5,1.59,2 Phone calls,160,churned,24,77187.17,69308.45
556,8815,2019-09-17,981565,531.725248,257,5,2.82,1 Email,0,active,24,40898.54,36808.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135827,2270627,2019-09-20,2752901,181.278905,451,2,1.61,2 Phone calls,160,active,24,114704.21,103073.79
135888,2271619,2019-09-20,3770135,730.646282,370,2,1.96,1 Email,0,active,24,157088.96,141380.06
135952,2272525,2019-09-09,1149944,310.963725,355,13,2.02,1 Visit,320,churned,24,47914.33,42802.90
135966,2272758,2019-09-20,5940504,157.506203,499,2,1.46,1 Visit,320,active,24,247521.00,222448.90


In [97]:
top_1_percent_spenders_df.drop(columns=['last_purchase', "avg_transaction_value",
                                        "total_spendings", "total_transactions",
                                        "months_active", "avg_monthly_spend",
                                        "cost"], inplace=True)
top_1_percent_spenders_df.to_csv('top_1_percent_customers.csv', index=False)

In [ ]:
top_1_percent_spenders_transactions.sort_values(by='date_order', inplace=True)
top_1_percent_spenders_transactions.to_csv('top_1_percent_transactions.csv', index=False)

C:\Users\charl\AppData\Local\Temp\ipykernel_9516\707855643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1_percent_spenders_transactions.sort_values(by='date_order', inplace=True)


In [31]:
print(transactions_dataset.shape[0])
print(top_1_percent_spenders_transactions.shape[0])

63319315
14786235


In [32]:
top_1_percent_spenders_df

,Unnamed: 0,client_id,last_purchase,total_monetary,avg_monetary,frequency,recency,avg_time_gap,dynamic_threshold,churner
1,1,14,2019-09-20,3103187,243.158351,505,2,1.438492,4.315476,False
37,38,753,2019-09-19,997376,196.954203,284,3,2.547703,7.643110,False
292,293,4696,2019-09-20,1152072,120.484416,492,2,1.476578,4.429735,False
434,435,6839,2019-09-17,1852492,161.578021,455,5,1.590308,4.770925,True
556,558,8815,2019-09-17,981565,531.725248,257,5,2.820312,8.460938,False
...,...,...,...,...,...,...,...,...,...,...
135827,136075,2270627,2019-09-20,2752901,181.278905,451,2,1.608889,4.826667,False
135888,136136,2271619,2019-09-20,3770135,730.646282,370,2,1.964770,5.894309,False
135952,136200,2272525,2019-09-09,1149944,310.963725,355,13,2.016949,6.050847,True
135966,136214,2272758,2019-09-20,5940504,157.506203,499,2,1.455823,4.367470,False


# 100K Budget Allocation

In [69]:
customer_summary = pd.read_csv('customer_summary.csv') 
#transactions_dataset = pd.read_csv('data/transactions_dataset.csv', sep=";")
relationships = pd.read_csv('data/sales_client_relationship_dataset.csv')
customer_summary = customer_summary.merge(relationships, on='client_id', how='left')
customer_summary["quali_relation"] = customer_summary["quali_relation"].map({"Agreeable client": "1 Email",
                                                                              "Demanding client": "2 Phone calls",
                                                                                "Difficult client": "1 Visit"})
customer_summary.rename(columns={"quali_relation": "effort_required"}, inplace=True)
customer_summary["cost"] = customer_summary["effort_required"].map({"1 Email": 0,
                                                                    "2 Phone calls": 160,
                                                                    "1 Visit": 320})

In [70]:
customer_summary = customer_summary.rename(columns={"total_monetary": "total_spendings",
                                                                      "frequency": "total_transactions",
                                                                      "avg_monetary": "avg_transaction_value",
                                                                      "recency": "days_since_last_transaction",
                                                                      "avg_time_gap": "avg_days_between_transactions"})

customer_summary['last_purchase'] = pd.to_datetime(customer_summary['last_purchase'])

# Calculate the number of months each customer has been active
customer_summary['months_active'] = (customer_summary['last_purchase'].dt.year - 2017) * 12 + customer_summary['last_purchase'].dt.month - 9

# Calculate the average monthly spend
customer_summary['avg_monthly_spend'] = customer_summary['total_spendings'] / customer_summary['months_active']

# Round the average monthly spend
customer_summary['avg_monthly_spend'] = customer_summary['avg_monthly_spend'].round(2)

customer_summary["potential_monthly_profit"] = customer_summary["avg_monthly_spend"] * 0.9 - (customer_summary["cost"])

customer_summary = customer_summary[customer_summary["avg_monthly_spend"]<10000000000]

In [71]:
customer_summary[customer_summary["churner"]==True]["potential_monthly_profit"].sum() / 1000000

np.float64(92.292584431)

In [72]:
customer_summary = customer_summary.sort_values("potential_monthly_profit", ascending=False)

In [73]:
churners = customer_summary[customer_summary["churner"]==True]
churners = churners[churners["total_transactions"]>10]

In [74]:
# Calculate the cumulative sum of the cost column
churners['cumulative_cost'] = churners['cost'].cumsum()

# Select the rows where the cumulative sum is less than or equal to 100,000
selected_churners_df = churners[(churners['cumulative_cost'] <= 128000) | (churners['cost']==0)]

# Drop the cumulative_cost column as it is no longer needed
#selected_churners_df = selected_churners_df.drop(columns=['cumulative_cost'])

selected_churners_df


,Unnamed: 0,client_id,last_purchase,total_spendings,avg_transaction_value,total_transactions,days_since_last_transaction,avg_days_between_transactions,dynamic_threshold,churner,effort_required,cost,months_active,avg_monthly_spend,potential_monthly_profit,cumulative_cost
132401,132640,2212698,2019-09-13,9853509,387.521503,439,9,1.639269,4.917808,True,2 Phone calls,160,24,410562.88,369346.592,160
35420,35502,596258,2019-09-13,6927660,141.138868,493,9,1.459350,4.378049,True,1 Email,0,24,288652.50,259787.250,160
70848,70996,1189655,2019-09-12,6464176,754.279570,458,10,1.568928,4.706783,True,1 Visit,320,24,269340.67,242086.603,480
82735,82899,1386568,2019-09-13,5924659,208.739713,455,9,1.581498,4.744493,True,1 Visit,320,24,246860.79,221854.711,800
102844,103046,1722816,2019-09-13,4660182,156.982471,468,9,1.537473,4.612420,True,1 Email,0,24,194174.25,174756.825,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52298,52410,880657,2019-07-02,892,6.280749,25,82,25.875000,77.625000,True,1 Email,0,22,40.55,36.495,2946240
119789,120015,2001273,2018-11-15,535,22.300417,11,311,39.800000,119.400000,True,1 Email,0,14,38.21,34.389,2951360
4810,4818,79066,2018-09-21,400,21.063158,12,366,30.545455,91.636364,True,1 Email,0,12,33.33,29.997,2959680
22044,22092,369189,2018-10-12,313,2.005147,17,345,16.875000,50.625000,True,1 Email,0,13,24.08,21.672,2976000


In [75]:
selected_churners_df["potential_monthly_profit"].sum()/1000000

np.float64(51.76064120200001)

In [68]:
selected_churners_df

,Unnamed: 0,client_id,last_purchase,total_spendings,avg_transaction_value,total_transactions,days_since_last_transaction,avg_days_between_transactions,dynamic_threshold,churner,effort_required,cost,months_active,avg_monthly_spend,potential_monthly_profit,cumulative_cost
132401,132640,2212698,2019-09-13,9853509,387.521503,439,9,1.639269,4.917808,True,2 Phone calls,160,24,410562.88,369346.592,160
35420,35502,596258,2019-09-13,6927660,141.138868,493,9,1.459350,4.378049,True,1 Email,0,24,288652.50,259787.250,160
70848,70996,1189655,2019-09-12,6464176,754.279570,458,10,1.568928,4.706783,True,1 Visit,320,24,269340.67,242086.603,480
82735,82899,1386568,2019-09-13,5924659,208.739713,455,9,1.581498,4.744493,True,1 Visit,320,24,246860.79,221854.711,800
102844,103046,1722816,2019-09-13,4660182,156.982471,468,9,1.537473,4.612420,True,1 Email,0,24,194174.25,174756.825,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52298,52410,880657,2019-07-02,892,6.280749,25,82,25.875000,77.625000,True,1 Email,0,22,40.55,36.495,2946240
119789,120015,2001273,2018-11-15,535,22.300417,11,311,39.800000,119.400000,True,1 Email,0,14,38.21,34.389,2951360
4810,4818,79066,2018-09-21,400,21.063158,12,366,30.545455,91.636364,True,1 Email,0,12,33.33,29.997,2959680
22044,22092,369189,2018-10-12,313,2.005147,17,345,16.875000,50.625000,True,1 Email,0,13,24.08,21.672,2976000


In [66]:
churners = churners[churners['potential_monthly_profit'] > 0]

In [67]:
churners

,Unnamed: 0,client_id,last_purchase,total_spendings,avg_transaction_value,total_transactions,days_since_last_transaction,avg_days_between_transactions,dynamic_threshold,churner,effort_required,cost,months_active,avg_monthly_spend,potential_monthly_profit,cumulative_cost
132401,132640,2212698,2019-09-13,9853509,387.521503,439,9,1.639269,4.917808,True,2 Phone calls,160,24,410562.88,369346.592,160
35420,35502,596258,2019-09-13,6927660,141.138868,493,9,1.459350,4.378049,True,1 Email,0,24,288652.50,259787.250,160
70848,70996,1189655,2019-09-12,6464176,754.279570,458,10,1.568928,4.706783,True,1 Visit,320,24,269340.67,242086.603,480
82735,82899,1386568,2019-09-13,5924659,208.739713,455,9,1.581498,4.744493,True,1 Visit,320,24,246860.79,221854.711,800
102844,103046,1722816,2019-09-13,4660182,156.982471,468,9,1.537473,4.612420,True,1 Email,0,24,194174.25,174756.825,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46340,46439,780915,2019-02-13,6055,86.498016,12,221,32.909091,98.727273,True,1 Visit,320,17,356.18,0.562,3023360
55721,55842,938610,2018-12-07,2674,74.290481,11,289,43.700000,131.100000,True,2 Phone calls,160,15,178.27,0.443,3023520
10062,10078,168508,2019-01-14,2850,129.543076,11,251,46.900000,140.700000,True,2 Phone calls,160,16,178.12,0.308,3023680
22436,22484,375736,2018-11-28,2492,53.025870,18,298,21.882353,65.647059,True,2 Phone calls,160,14,178.00,0.200,3023840
